<a href="https://colab.research.google.com/github/LeraDoroshenko/mn/blob/main/%D0%9B%D0%B0%D0%B14_2_%D0%94%D0%BE%D1%80%D0%BE%D1%88%D0%B5%D0%BD%D0%BA%D0%BE_%D0%92_4_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Завантаження датасету Titanic
df_titanic = pd.read_csv('/content/drive/My Drive/Colab Notebooks/titanic (2).csv')

# Перегляд перших кількох рядків
print(df_titanic.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   PassengerId  Survived  Pclass  \
0          892         0       3   
1          893         1       3   
2          894         0       2   
3          895         0       3   
4          896         1       3   

                                           Name     Sex   Age  SibSp  Parch  \
0                              Kelly, Mr. James    male  34.5      0      0   
1              Wilkes, Mrs. James (Ellen Needs)  female  47.0      1      0   
2                     Myles, Mr. Thomas Francis    male  62.0      0      0   
3                              Wirz, Mr. Albert    male  27.0      0      0   
4  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female  22.0      1      1   

    Ticket     Fare Cabin Embarked  
0   330911   7.8292   NaN        Q  
1   363272   7.0000   NaN        S  
2   240276   9.6875   NaN        Q  
3   315154   8.6625   NaN      

In [10]:
# Перевірка назви стовпців та типів даних
print(df_titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    int64  
 2   Pclass       418 non-null    int64  
 3   Name         418 non-null    object 
 4   Sex          418 non-null    object 
 5   Age          332 non-null    float64
 6   SibSp        418 non-null    int64  
 7   Parch        418 non-null    int64  
 8   Ticket       418 non-null    object 
 9   Fare         417 non-null    float64
 10  Cabin        91 non-null     object 
 11  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB
None


In [11]:
# Перевірка наявності пропущених значень
print(df_titanic.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [12]:
# Розмір даних
print(df_titanic.shape)

(418, 12)


In [13]:
from sklearn.model_selection import train_test_split

X_titanic = df_titanic.drop('Survived', axis=1)
y_titanic = df_titanic['Survived']

# Видалення пропущених значень та обробка даних
df_titanic.dropna(subset=['Age', 'Embarked'], inplace=True)
df_titanic['Sex'] = df_titanic['Sex'].map({'male': 0, 'female': 1})
df_titanic['Embarked'] = df_titanic['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

# Визначення X і y
X_titanic = df_titanic[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y_titanic = df_titanic['Survived']

# Розділення на навчальний і тестовий набори
X_train_titanic, X_test_titanic, y_train_titanic, y_test_titanic = train_test_split(X_titanic, y_titanic, test_size=0.2, random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler

# Масштабування даних
scaler = StandardScaler()
X_train_titanic = scaler.fit_transform(X_train_titanic)
X_test_titanic = scaler.transform(X_test_titanic)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X_train_titanic = imputer.fit_transform(X_train_titanic)
X_test_titanic = imputer.transform(X_test_titanic)

# Моделі
log_reg = LogisticRegression()
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier()

# Параметри для GridSearch
param_grid_log_reg = {'C': [0.01, 0.1, 1, 10, 100]}
param_grid_decision_tree = {'max_depth': [3, 5, 7, 10], 'min_samples_split': [2, 5, 10]}
param_grid_random_forest = {'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 7, 10]}

# Налаштування GridSearchCV
grid_log_reg = GridSearchCV(log_reg, param_grid_log_reg, cv=5)
grid_decision_tree = GridSearchCV(decision_tree, param_grid_decision_tree, cv=5)
grid_random_forest = GridSearchCV(random_forest, param_grid_random_forest, cv=5)

# Тренування моделей
grid_log_reg.fit(X_train_titanic, y_train_titanic)
grid_decision_tree.fit(X_train_titanic, y_train_titanic)
grid_random_forest.fit(X_train_titanic, y_train_titanic)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 5, 7, 10],
                         'n_estimators': [10, 50, 100]})

In [18]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Прогнози для тестової вибірки
y_pred_log_reg = grid_log_reg.predict(X_test_titanic)
y_pred_decision_tree = grid_decision_tree.predict(X_test_titanic)
y_pred_random_forest = grid_random_forest.predict(X_test_titanic)

# Точність моделей
print("Точність Логістичної регресії:", accuracy_score(y_test_titanic, y_pred_log_reg))
print("Точність Дерева рішень:", accuracy_score(y_test_titanic, y_pred_decision_tree))
print("Точність Випадкового лісу:", accuracy_score(y_test_titanic, y_pred_random_forest))

# Матриця помилок і звіти
print("Матриця помилок Логістичної регресії:\n", confusion_matrix(y_test_titanic, y_pred_log_reg))
print("Звіт про класифікацію Логістичної регресії:\n", classification_report(y_test_titanic, y_pred_log_reg))

print("Матриця помилок Дерева рішень:\n", confusion_matrix(y_test_titanic, y_pred_decision_tree))
print("Звіт про класифікацію Дерева рішень:\n", classification_report(y_test_titanic, y_pred_decision_tree))

print("Матриця помилок Випадкового лісу:\n", confusion_matrix(y_test_titanic, y_pred_random_forest))
print("Звіт про класифікацію Випадкового лісу:\n", classification_report(y_test_titanic, y_pred_random_forest))

Точність Логістичної регресії: 1.0
Точність Дерева рішень: 1.0
Точність Випадкового лісу: 1.0
Матриця помилок Логістичної регресії:
 [[42  0]
 [ 0 25]]
Звіт про класифікацію Логістичної регресії:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        42
           1       1.00      1.00      1.00        25

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67

Матриця помилок Дерева рішень:
 [[42  0]
 [ 0 25]]
Звіт про класифікацію Дерева рішень:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        42
           1       1.00      1.00      1.00        25

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67

Матриця помилок Випадкового лісу:
 [[42  0]
 [ 0 25]]
Звіт п

In [19]:
# Прогноз для перших 10 записів тестової вибірки
predictions = grid_random_forest.predict(X_test_titanic[:10])
print("Прогноз для 10 випадків:", predictions)

Прогноз для 10 випадків: [0 0 0 1 0 0 1 1 1 0]
